# <center> CS F320 Foundations of Data Science <center>

## <center> Assignment <center>

***

### Group 8

#### 1. 2022A7PS0145P - Armaan Gupta
#### 2. 2022A7PS0065P - Animish Tiwari
#### 3. 2022A7PS0164P - Anjaneya Bajaj
#### 4. 2022A7PS0120P - Aryan Jain

***

#### Dataset : [Electrical Grid Stability Simulated Data](https://archive.ics.uci.edu/dataset/471/electrical+grid+stability+simulated+data)

In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, LabelEncoder
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, accuracy_score, precision_score, recall_score, f1_score, silhouette_score, pairwise_distances, confusion_matrix
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, AdaBoostRegressor
from xgboost import XGBRegressor
from ngboost import NGBRegressor
from tabulate import tabulate
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
from sklearn_extra.cluster import KMedoids
from scipy.spatial.distance import pdist, squareform, cdist
from scipy.stats import pearsonr
from ucimlrepo import fetch_ucirepo 
import optuna
import os

## Loading in the dataset and pre-processing

- We will first load in the dataset and take a look at the first few rows to understand the data better.
- We will then check for any missing values - there are none.
- As per the question, we will drop categorical columns - there are None.
- Eliminating outliers - any value that is more than 3 standard deviations away from the mean will be considered an outlier and will be removed.
- Making plots to visualize the data distributions.
- Seperating the two target variables one for classification and one for regression.
- Splitting the data into training and testing sets.
- Applying PCA, taking 10 principal components to account for >95% variance.

In [ ]:
# fetch dataset 
electrical_grid_stability_simulated_data = fetch_ucirepo(id=471) 
  
# data (as pandas dataframes) 
X = electrical_grid_stability_simulated_data.data.features 
y = electrical_grid_stability_simulated_data.data.targets 
  
# metadata 
print(electrical_grid_stability_simulated_data.metadata) 
  
# variable information 
print(electrical_grid_stability_simulated_data.variables) 


In [ ]:
df = pd.concat([X, y], axis=1)

In [ ]:
df.describe()

In [ ]:
df.info()

In [ ]:
# Plotting the distribution of all numeric columns

numeric_columns = df.drop(columns=['stab', 'stabf']).columns

num_plots = len(numeric_columns)
num_cols = 3
num_rows = (num_plots // num_cols) + (1 if num_plots % num_cols != 0 else 0) 

fig, axes = plt.subplots(num_rows, num_cols, figsize=(16, 6*num_rows))
axes = axes.flatten()  

for i, col in enumerate(numeric_columns):
    sns.histplot(df[col], kde=True, ax=axes[i])
    axes[i].set_title(f'Distribution of {col}')

plt.tight_layout()
plt.show()


In [ ]:
# Plot for variable 'stab'

sns.histplot(df['stab'], kde=True)
plt.title('Distribution of stab')
plt.xlabel('stab')
plt.show()

In [ ]:
# Plot for variable 'stabf'

sns.countplot(x='stabf', data=df)
plt.title('Distribution of stabf')
plt.show()

In [ ]:
# Splitting the data into features and target

X = df.drop(['stab', 'stabf'], axis=1)
y = df['stab']
y_classification = df['stabf']

In [ ]:
# Removing outliers in all columns

X = X[(np.abs(stats.zscore(X)) < 3).all(axis=1)] 
y = y[(np.abs(stats.zscore(y)) < 3)]

In [ ]:
# Creating the train and test sets

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train_classification, X_test_classification, y_train_classification, y_test_classification = train_test_split(X, y_classification, test_size=0.2, random_state=42)

In [ ]:
# Initialize label encoder
label_encoder = LabelEncoder()

# Fit and transform the labels
y_train_classification_encoded = label_encoder.fit_transform(y_train_classification)
y_test_classification_encoded = label_encoder.transform(y_test_classification)

In [ ]:
for i, class_ in enumerate(label_encoder.classes_):
    print(f'{class_} -> {i}')

In [ ]:
# Z-Score transformation

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# PCA 

# pca = PCA(n_components=3)
pca = PCA()
X_train_pca = pca.fit_transform(X_train_scaled)
X_test_pca = pca.transform(X_test_scaled)

In [ ]:
# Z-Score transformation

scaler = StandardScaler()
X_train_classification_scaled = scaler.fit_transform(X_train_classification)
X_test_classification_scaled = scaler.transform(X_test_classification)

# PCA 

# pca = PCA(n_components=3)
pca = PCA()
X_train_classification_pca = pca.fit_transform(X_train_classification_scaled)
X_test_classification_pca = pca.transform(X_test_classification_scaled)

In [ ]:
# Explained variance ratio

explained_variance = pca.explained_variance_ratio_

# Plotting scree plot
plt.figure(figsize=(8, 6))
plt.plot(range(1, len(explained_variance) + 1), explained_variance, marker='o', linestyle='--')
plt.title('Scree Plot')
plt.xlabel('Principal Component')
plt.ylabel('Explained Variance Ratio')
plt.show()

In [ ]:
cumulative_variance = pca.explained_variance_ratio_.cumsum()

plt.figure(figsize=(8, 6))
plt.plot(range(1, len(cumulative_variance) + 1), cumulative_variance, marker='o', linestyle='--')
plt.title('Cumulative Explained Variance')
plt.xlabel('Number of Principal Components')
plt.ylabel('Cumulative Explained Variance')
plt.axhline(y=0.95, color='r', linestyle='--', label='95% Variance')
plt.legend()
plt.show()


In [ ]:
# Based on plots, extracting 10 principal components

X_train_pca = X_train_pca[:, :10]
X_test_pca = X_test_pca[:, :10]

X_train_classification_pca = X_train_classification_pca[:, :10]
X_test_classification_pca = X_test_classification_pca[:, :10]

## Regression Models

Applying various regression models and calculating RMSE and MAE for each model. Models applied :
- Linear Regression
- Random Forests
- Extra Random Trees
- AdaBoost
- XGBoost
- NGBoost
- Neural Network

In [ ]:
models = [
            "Linear Regression",
            "Ridge Regression",
            "Lasso Regression",
            "Elastic Net",
            "Random Forest",
            "Extra Trees",
            "AdaBoost",
            "XGBoost",
            "NGBoost"
         ]

# Objective function for Optuna hyperparameter tuning
def objective(trial, model_name, X_train, y_train):
    if model_name == "Linear Regression":
        model = LinearRegression()
    elif model_name == "Ridge Regression":
        model = Ridge(
            alpha=trial.suggest_float("alpha", 1e-4, 100.0, log=True),
            max_iter=trial.suggest_int("max_iter", 100, 2000)
        )
    elif model_name == "Lasso Regression":
        model = Lasso(
            alpha=trial.suggest_float("alpha", 1e-4, 100.0, log=True),  
            max_iter=trial.suggest_int("max_iter", 100, 2000)           
        )
    elif model_name == "Elastic Net":
        model = ElasticNet(
            alpha=trial.suggest_float("alpha", 1e-4, 100.0, log=True),  
            l1_ratio=trial.suggest_float("l1_ratio", 0.0, 1.0),         
            max_iter=trial.suggest_int("max_iter", 100, 2000)           
        )
    elif model_name == "Random Forest":
        model = RandomForestRegressor(
            n_estimators=trial.suggest_int("n_estimators", 50, 500),
            max_depth=trial.suggest_int("max_depth", 3, 20),
            min_samples_split=trial.suggest_int("min_samples_split", 2, 20),
            min_samples_leaf=trial.suggest_int("min_samples_leaf", 1, 20),
            random_state=42,
        )
    elif model_name == "Extra Trees":
        model = ExtraTreesRegressor(
            n_estimators=trial.suggest_int("n_estimators", 50, 500),
            max_depth=trial.suggest_int("max_depth", 3, 20),
            min_samples_split=trial.suggest_int("min_samples_split", 2, 20),
            min_samples_leaf=trial.suggest_int("min_samples_leaf", 1, 20),
            random_state=42,
        )
    elif model_name == "AdaBoost":
        model = AdaBoostRegressor(
            n_estimators=trial.suggest_int("n_estimators", 50, 500),
            learning_rate=trial.suggest_float("learning_rate", 0.001, 1.0),
            random_state=42,
        )
    elif model_name == "XGBoost":
        model = XGBRegressor(
            n_estimators=trial.suggest_int("n_estimators", 50, 500),
            max_depth=trial.suggest_int("max_depth", 3, 20),
            learning_rate=trial.suggest_float("learning_rate", 0.01, 1),
            colsample_bytree=trial.suggest_float("colsample_bytree", 0.5, 1.0),
            random_state=42,
            objective='reg:squarederror',
        )
    elif model_name == "NGBoost":
        model = NGBRegressor(
            n_estimators=trial.suggest_int("n_estimators", 50, 500),
            learning_rate=trial.suggest_float("learning_rate", 0.01, 0.3),
            random_state=42,
        )
    else:
        raise ValueError(f"Unknown model: {model_name}")

    # Cross-validation to evaluate performance
    scores = cross_val_score(model, X_train, y_train, cv=5, scoring="neg_mean_squared_error")
    return np.sqrt(-scores.mean())

# Function to tune models using Optuna
def tune_model(model_name, X_train, y_train):
    study = optuna.create_study(direction="minimize")
    study.optimize(lambda trial: objective(trial, model_name, X_train, y_train), n_trials=1)
    return study.best_params, study.best_value

# Function to train models with the best hyperparameters
def train_model_with_best_params(model_name, best_params, X_train, y_train):
    if model_name == "Linear Regression":
        model = LinearRegression()
    elif model_name == "Ridge Regression":
        model = Ridge(**best_params)
    elif model_name == "Lasso Regression":
        model = Lasso(**best_params)
    elif model_name == "Elastic Net":
        model = ElasticNet(**best_params)
    elif model_name == "Random Forest":
        model = RandomForestRegressor(**best_params)
    elif model_name == "Extra Trees":
        model = ExtraTreesRegressor(**best_params)
    elif model_name == "AdaBoost":
        model = AdaBoostRegressor(**best_params)
    elif model_name == "XGBoost":
        model = XGBRegressor(**best_params)
    elif model_name == "NGBoost":
        model = NGBRegressor(**best_params)
    else:
        raise ValueError(f"Unknown model: {model_name}")
    
    print(f"Training {model_name} with best hyperparameters...")
    print(f"Best hyperparameters: {best_params}")

    model.fit(X_train, y_train)
    return model

def plot_predicted_vs_actual(y_pred, y_test, model_name, image_path):
    plt.figure(figsize=(8, 8))
    plt.scatter(
        y_test, y_pred, 
        alpha=0.6, color="blue", marker='o', label="Predicted Values"
    )

    plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 
             "r--", label="Ideal: Predicted = Actual")

    plt.title(f"Predicted vs Actual Values: {model_name}")
    plt.xlabel("Actual Values")
    plt.ylabel("Predicted Values")
    plt.legend()
    plt.grid()

    if os.path.exists(image_path) == False:
        os.makedirs(image_path)
    plot_name = os.path.join(image_path, f"{model_name}_regression.png")
    plt.savefig(plot_name)
    print(f"Predicted vs Actual plot saved as: {plot_name}")
    plt.close()

# Train models with optimized parameters and evaluate them
def regression_models_optimized(X_train, X_test, y_train, y_test, models, image_path):
    model_results = []
    model_params = []
    for model_name in models:
        print(f"Tuning hyperparameters for {model_name}...")
        best_params, best_score = tune_model(model_name, X_train, y_train)
        print(f"Best parameters for {model_name}: {best_params}")
        
        # Train the model with the best parameters
        tuned_model = train_model_with_best_params(model_name, best_params, X_train, y_train)
        
        # Evaluate the model
        y_pred = tuned_model.predict(X_test)
        
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        mae = mean_absolute_error(y_test, y_pred)
        
        plot_predicted_vs_actual(y_pred, y_test, model_name, image_path)

        model_results.append({"Model": model_name, "RMSE": rmse, "MAE": mae})
        model_params.append({"Model": model_name, "Best Parameters": best_params})
    
    results_df = pd.DataFrame(model_results)
    params_df = pd.DataFrame(model_params)
    return results_df, params_df


In [ ]:
# Train and evaluate models with optimized hyperparameters
results_original, params_original = regression_models_optimized(X_train, X_test, y_train, y_test, models, "./original_regression_plots")

In [ ]:
results_pca, params_pca = regression_models_optimized(X_train_pca, X_test_pca, y_train, y_test, models, "./pca_regression_plots")

In [ ]:
print("Regression Models Evaluation on Original Dataset\n")
print(tabulate(results_original, headers=["Model", "RMSE", "MAE"], tablefmt="grid"))
print("\nBest Hyperparameters for Original Dataset\n")
print(tabulate(params_original, headers="keys", tablefmt="grid"))

In [ ]:
print("Regresssion Models Evaluation on PCA Dataset\n")
print(tabulate(results_pca, headers=["Model", "RMSE", "MAE"], tablefmt="grid"))
print("\nBest Hyperparameters for PCA Dataset\n")
print(tabulate(params_pca, headers="keys", tablefmt="grid"))

## Classification Models

Applying various classification models and calculating accuracy, precision, recall and F1-score for each model. Models applied :
- Logistic Regression
- Naive Bayes
- KNN
- Linear SVM
- Kernel SVM
- Decision Trees
- 2 Layer Fully Connected Neural Network

In [ ]:
models = [
            "Logistic Regression with L1 Regularization",
            "Logistic Regression with L2 Regularization",
            "Logistic Regression with Elastic Net Regularization",
            "Gaussian Naive Bayes",
            "K-Nearest Neighbors",
            "Linear SVM",
            "Kernel SVM",
            "Decision Tree"
         ]

# Objective function for Optuna hyperparameter tuning (Classification)
def objective(trial, model_name, X_train, y_train):
    if model_name == "Logistic Regression with L1 Regularization":
        model = LogisticRegression(
            penalty="l1",
            solver="liblinear",  # 'liblinear' supports L1 regularization
            C=trial.suggest_float("C", 1e-4, 10.0, log=True),
            max_iter=trial.suggest_int("max_iter", 100, 2000),
            random_state=42,
        )
    elif model_name == "Logistic Regression with L2 Regularization":
        model = LogisticRegression(
            penalty="l2",
            solver="lbfgs",
            C=trial.suggest_float("C", 1e-4, 10.0, log=True),
            max_iter=trial.suggest_int("max_iter", 100, 2000),
            random_state=42,
        )
    elif model_name == "Logistic Regression with Elastic Net Regularization":
        model = LogisticRegression(
            penalty="elasticnet",
            solver="saga",  # 'saga' supports Elastic Net
            C=trial.suggest_float("C", 1e-4, 10.0, log=True),
            l1_ratio=trial.suggest_float("l1_ratio", 0.0, 1.0),
            max_iter=trial.suggest_int("max_iter", 100, 2000),
            random_state=42,
        )
    elif model_name == "Gaussian Naive Bayes":
        model = GaussianNB()
    elif model_name == "K-Nearest Neighbors":
        model = KNeighborsClassifier(
            n_neighbors=trial.suggest_int("n_neighbors", 1, 50),
            weights=trial.suggest_categorical("weights", ["uniform", "distance"]),
            p=trial.suggest_int("p", 1, 2),  # Minkowski metric: 1 for Manhattan, 2 for Euclidean
        )
    elif model_name == "Linear SVM":
        model = SVC(
            kernel="linear",
            C=trial.suggest_float("C", 1e-4, 10.0, log=True),
            random_state=42,
        )
    elif model_name == "Kernel SVM":
        model = SVC(
            kernel="rbf",
            C=trial.suggest_float("C", 1e-4, 10.0, log=True),
            gamma=trial.suggest_float("gamma", 1e-4, 10.0, log=True),
            random_state=42,
        )
    elif model_name == "Decision Tree":
        model = DecisionTreeClassifier(
            max_depth=trial.suggest_int("max_depth", 1, 20),
            min_samples_split=trial.suggest_int("min_samples_split", 2, 20),
            min_samples_leaf=trial.suggest_int("min_samples_leaf", 1, 20),
            random_state=42,
        )
    else:
        raise ValueError(f"Unknown model: {model_name}")

    # Cross-validation to evaluate performance
    scores = cross_val_score(model, X_train, y_train, cv=5, scoring="accuracy")
    return scores.mean()

# Function to tune models using Optuna
def tune_model(model_name, X_train, y_train):
    study = optuna.create_study(direction="maximize")
    study.optimize(lambda trial: objective(trial, model_name, X_train, y_train), n_trials=1)
    return study.best_params, study.best_value

# Function to train models with the best hyperparameters
def train_model_with_best_params(model_name, best_params, X_train, y_train):
    if model_name == "Logistic Regression with L1 Regularization":
        model = LogisticRegression(penalty="l1", solver="liblinear", random_state=42, **best_params)
    elif model_name == "Logistic Regression with L2 Regularization":
        model = LogisticRegression(penalty="l2", solver="lbfgs", random_state=42, **best_params)
    elif model_name == "Logistic Regression with Elastic Net Regularization":
        model = LogisticRegression(penalty="elasticnet", solver="saga", random_state=42, **best_params)
    elif model_name == "Gaussian Naive Bayes":
        model = GaussianNB()
    elif model_name == "K-Nearest Neighbors":
        model = KNeighborsClassifier(**best_params)
    elif model_name == "Linear SVM":
        model = SVC(kernel="linear", random_state=42, **best_params)
    elif model_name == "Kernel SVM":
        model = SVC(kernel="rbf", random_state=42, **best_params)
    elif model_name == "Decision Tree":
        model = DecisionTreeClassifier(random_state=42, **best_params)
    else:
        raise ValueError(f"Unknown model: {model_name}")
    
    print(f"Training {model_name} with best hyperparameters...")
    print(f"Best hyperparameters: {best_params}")

    model.fit(X_train, y_train)
    return model

def plot_confusion_matrix(y_pred, y_test, model_name, image_path):
    cm = confusion_matrix(y_test, y_pred)
    
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", cbar=False)
    plt.title(f"Confusion Matrix: {model_name}")
    plt.xlabel("Predicted")
    plt.ylabel("Actual")

    if os.path.exists(image_path) == False:
        os.makedirs(image_path)
    plot_name = os.path.join(image_path, f"{model_name}_confusion_matrix.png")
    plt.savefig(plot_name)
    print(f"Confusion matrix saved as: {plot_name}")
    plt.close()
    
# Train models with optimized parameters and evaluate them
def classification_models_optimized(X_train, X_test, y_train, y_test, models, image_path):
    model_results = []
    model_params = []
    for model_name in models:
        print(f"Tuning hyperparameters for {model_name}...")
        best_params, best_score = tune_model(model_name, X_train, y_train)
        print(f"Best parameters for {model_name}: {best_params}")
        
        # Train the model with the best parameters
        tuned_model = train_model_with_best_params(model_name, best_params, X_train, y_train)
        
        # Evaluate the model
        y_pred = tuned_model.predict(X_test)
        
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred, average="weighted")
        recall = recall_score(y_test, y_pred, average="weighted")
        f1 = f1_score(y_test, y_pred, average="weighted")

        plot_confusion_matrix(y_pred, y_test, model_name, image_path)
        model_results.append({"Model": model_name, "Accuracy": accuracy, "Precision": precision, "Recall": recall, "F1-Score": f1})
        model_params.append({"Model": model_name, "Best Parameters": best_params})
    
    results_df = pd.DataFrame(model_results)
    params_df = pd.DataFrame(model_params)
    return results_df, params_df

In [ ]:
results_original, params_original = classification_models_optimized(X_train_classification, X_test_classification, y_train_classification, y_test_classification, models, "./original_classification_plots")

In [ ]:
results_pca, params_pca = classification_models_optimized(X_train_classification_pca, X_test_classification_pca, y_train_classification, y_test_classification, models, "./pca_classification_plots")

In [ ]:
class BinaryNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(BinaryNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, 1)
        # self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        # return self.sigmoid(x)
        return x

def train_model(model, X_train, y_train, criterion, optimizer, batch_size, num_epochs):
    model.train()
    dataset = torch.utils.data.TensorDataset(X_train, y_train)
    data_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)
    
    loss_history = [] 
    accuracy_history = [] 
    
    for epoch in range(num_epochs):
        epoch_loss = 0.0
        correct = 0
        total = 0
        
        for batch_X, batch_y in data_loader:
            # Forward pass
            outputs = model(batch_X)
            loss = criterion(outputs.squeeze(), batch_y.float())
            
            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            epoch_loss += loss.item() * batch_X.size(0)  # Accumulate total loss
            predictions = (torch.sigmoid(outputs.squeeze()) > 0.5).long()
            correct += (predictions == batch_y).sum().item()
            total += batch_y.size(0)
        
        # Calculate average loss and accuracy for the epoch
        avg_loss = epoch_loss / len(data_loader.dataset)
        accuracy = correct / total
        
        loss_history.append(avg_loss)
        accuracy_history.append(accuracy)
        
        if (epoch + 1) % 10 == 0:
            print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}, Accuracy: {accuracy:.4f}")
    
    return loss_history, accuracy_history

def plot_loss_vs_epochs(loss_history, model_name, image_path):
    plt.figure(figsize=(8, 6))
    plt.plot(range(1, len(loss_history) + 1), loss_history, label="Loss", color="blue")
    plt.title(f"Loss vs Epochs for {model_name}")
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.legend()

    if os.path.exists(image_path) == False:
        os.makedirs(image_path)
    plot_name = os.path.join(image_path, f"{model_name}_loss_vs_epochs.png")
    plt.savefig(plot_name)
    plt.close()

def plot_accuracy_vs_epochs(accuracy_history, model_name, image_path):
    plt.figure(figsize=(8, 6))
    plt.plot(range(1, len(accuracy_history) + 1), accuracy_history, label="Accuracy", color="green")
    plt.title(f"Accuracy vs Epochs for {model_name}")
    plt.xlabel("Epochs")
    plt.ylabel("Accuracy")
    plt.legend()

    if os.path.exists(image_path) == False:
        os.makedirs(image_path)
    plot_name = os.path.join(image_path, f"{model_name}_accuracy_vs_epochs.png")
    plt.savefig(plot_name)
    plt.close()

def test_model(model, X_test, y_test, image_path, make_plot=False):
    model.eval()
    with torch.no_grad():
        # y_pred = (model(X_test).squeeze() > 0.5).numpy()
        logits = model(X_test).squeeze()  # Get logits
        probabilities = torch.sigmoid(logits)  # Convert to probabilities
        y_pred = (probabilities > 0.5).numpy()  # Threshold to get binary predictions
    
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    
    if (make_plot) :
        plot_confusion_matrix(y_pred, y_test, "Neural Network", image_path)

    return accuracy, precision, recall, f1

def objective(trial, X_train, y_train, X_test, y_test, X_train_tensor, y_train_tensor, X_test_tensor, y_test_tensor):
    input_size = X_train.shape[1]
    hidden_size = trial.suggest_int("hidden_size", 16, 128, step=16)
    learning_rate = trial.suggest_float("lr", 1e-4, 1e-1, log=True)
    batch_size = trial.suggest_categorical("batch_size", [16, 32, 64, 128])
    num_epochs = trial.suggest_int("num_epochs", 50, 300, step=50)
    
    # Initialize the model, criterion, and optimizer
    model = BinaryNN(input_size, hidden_size)
    # criterion = nn.BCELoss()
    criterion = nn.BCEWithLogitsLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    
    # Train the model
    train_model(model, X_train_tensor, y_train_tensor, criterion, optimizer, batch_size, num_epochs)
    
    # Test the model
    accuracy, _, _, _ = test_model(model, X_test_tensor, y_test_tensor)
    
    return accuracy

def tune_nn(X_train, y_train, X_test, y_test, X_train_tensor, y_train_tensor, X_test_tensor, y_test_tensor, results, best_params_df, image_path):
    study = optuna.create_study(direction="maximize")
    study.optimize(lambda trial: objective(trial, X_train, y_train, X_test, y_test, X_train_tensor, y_train_tensor, X_test_tensor, y_test_tensor), n_trials=1)
    
    best_params = study.best_params
    print("Best hyperparameters:", best_params)
    
    # Train the best model
    input_size = X_train.shape[1]
    model = BinaryNN(input_size, best_params["hidden_size"])
    criterion = nn.BCEWithLogitsLoss()
    optimizer = optim.Adam(model.parameters(), lr=best_params["lr"])
    
    # Train and evaluate the model
    loss_history, accuracy_history = train_model(model, X_train_tensor, y_train_tensor, criterion, optimizer, best_params["batch_size"], best_params["num_epochs"])
    accuracy, precision, recall, f1 = test_model(model, X_test_tensor, y_test_tensor, image_path, make_plot=True)
    
    plot_loss_vs_epochs(loss_history, "Neural Network", image_path)
    plot_accuracy_vs_epochs(accuracy_history, "Neural Network", image_path)
    
    print("Final Model Metrics:")
    print(f"Accuracy: {accuracy:.2f}")
    print(f"Precision: {precision:.2f}")
    print(f"Recall: {recall:.2f}")
    print(f"F1-Score: {f1:.2f}")
    
    results.loc[len(results)] = ["Neural Network", accuracy, precision, recall, f1]
    best_params_df.loc[len(best_params_df)] = ["Neural Network", best_params]
    
    return model

In [ ]:
# Convert data to tensors
X_train_tensor = torch.tensor(X_train_classification.values, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train_classification_encoded, dtype=torch.long)
X_test_tensor = torch.tensor(X_test_classification.values, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test_classification_encoded, dtype=torch.long)

tune_nn(X_train_classification, y_train_classification, X_test_classification, y_test_classification, X_train_tensor, y_train_tensor, X_test_tensor, y_test_tensor, results_original, params_original, "./original_classification_plots")

In [ ]:
# Convert data to tensors
X_train_tensor = torch.tensor(X_train_classification_pca, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train_classification_encoded, dtype=torch.long)
X_test_tensor = torch.tensor(X_test_classification_pca, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test_classification_encoded, dtype=torch.long)

tune_nn(X_train_classification_pca, y_train_classification, X_test_classification_pca, y_test_classification, X_train_tensor, y_train_tensor, X_test_tensor, y_test_tensor, results_pca, params_pca, "./pca_classification_plots")

In [ ]:
print("Classification Models Evaluation on Original Dataset\n")
print(tabulate(results_original, headers=["Model", "Accuracy", "Precision", "Recall", "F1-Score"], tablefmt="grid"))
print("\nBest Hyperparameters for Original Dataset\n")
print(tabulate(params_original, headers="keys", tablefmt="grid"))

In [ ]:
print("Classification Models Evaluation on PCA Dataset\n")
print(tabulate(results_pca, headers=["Model", "Accuracy", "Precision", "Recall", "F1-Score"], tablefmt="grid"))
print("\nBest Hyperparameters for PCA Dataset\n")
print(tabulate(params_pca, headers="keys", tablefmt="grid"))

## Clustering Models

Scaling the original data.
Applying various clustering models and calculating SSE, Silhouette, BetaCV, Dunn’s Index and Hubert’s Statistic for each model. Models applied :
- KMeans
- EM Clustering
- KMedoids

In [ ]:
# Scaling the data 

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [ ]:
def k_means_clustering(X, k):
    kmeans = KMeans(n_clusters=k, init="k-means++", random_state=42)
    kmeans.fit(X)

    return kmeans

def em_clustering(X, k):
    gmm = GaussianMixture(n_components=k, random_state=42)
    gmm.fit(X)

    return gmm

def k_medoids_clustering(X, k):
    kmedoids = KMedoids(n_clusters=k, random_state=42)
    kmedoids.fit(X)

    return kmedoids

In [ ]:
def compute_sse(X, model):
    if isinstance(model, KMeans):
        labels = model.labels_
        centroids = model.cluster_centers_
        sse = np.sum((X - centroids[labels]) ** 2)
    elif isinstance(model, GaussianMixture):
        labels = model.predict(X)
        centroids = model.means_
        sse = np.sum((X - centroids[labels]) ** 2)
    elif isinstance(model, KMedoids):
        labels = model.labels_
        medoids = model.cluster_centers_
        sse = np.sum((X - medoids[labels]) ** 2)
    return sse

def compute_silhouette(X, model):
    if isinstance(model, KMeans):
        labels = model.labels_
    elif isinstance(model, GaussianMixture):
        labels = model.predict(X)
    elif isinstance(model, KMedoids):
        labels = model.labels_
    
    return silhouette_score(X, labels)

# def compute_dunn_index(X, model):
#     if isinstance(model, KMeans):
#         labels = model.labels_
#     elif isinstance(model, GaussianMixture):
#         labels = model.predict(X)
#     elif isinstance(model, KMedoids):
#         labels = model.labels_

#     # Compute pairwise distances between points
#     pairwise_dists = pairwise_distances(X)

#     # Calculate intra-cluster distances (min distance within the same cluster)
#     intra_cluster_distances = []
#     for label in np.unique(labels):
#         cluster_points = X[labels == label]
#         if len(cluster_points) > 1:
#             intra_cluster_distances.append(np.min(pairwise_distances(cluster_points)))
#     min_intra_cluster_distance = np.min(intra_cluster_distances)

#     # Calculate inter-cluster distances (min distance between different clusters)
#     inter_cluster_distances = []
#     for i, label1 in enumerate(np.unique(labels)):
#         for j, label2 in enumerate(np.unique(labels)):
#             if label1 != label2:
#                 cluster1_points = X[labels == label1]
#                 cluster2_points = X[labels == label2]
#                 inter_cluster_distances.append(np.min(pairwise_distances(np.concatenate([cluster1_points, cluster2_points]))))
#     max_inter_cluster_distance = np.max(inter_cluster_distances)

#     return min_intra_cluster_distance / max_inter_cluster_distance

# def compute_dunn_index(X, model):
#     # Get labels based on the model type
#     if isinstance(model, KMeans):
#         labels = model.labels_
#     elif isinstance(model, GaussianMixture):
#         labels = model.predict(X)
#     elif isinstance(model, KMedoids):
#         labels = model.labels_

#     # Compute pairwise distances between points
#     pairwise_dists = pairwise_distances(X)

#     # Calculate intra-cluster distances (min distance within the same cluster)
#     intra_cluster_distances = []
#     for label in np.unique(labels):
#         cluster_points = X[labels == label]
#         if len(cluster_points) > 1:
#             # Compute pairwise distances within the cluster
#             cluster_pairwise_dists = pairwise_distances(cluster_points)
#             np.fill_diagonal(cluster_pairwise_dists, np.inf)  # Ignore the diagonal (distance to itself)
#             min_intra_cluster_distance = np.min(cluster_pairwise_dists)
#             intra_cluster_distances.append(min_intra_cluster_distance)
#         else:
#             # For a cluster with only one point, intra-cluster distance is 0
#             intra_cluster_distances.append(0)
    
#     # If no intra-cluster distances were computed (i.e., all clusters are singletons), return NaN
#     if not intra_cluster_distances:
#         return np.nan
    
#     min_intra_cluster_distance = np.min(intra_cluster_distances)

#     # Calculate inter-cluster distances (min distance between different clusters)
#     inter_cluster_distances = []
#     for i, label1 in enumerate(np.unique(labels)):
#         for j, label2 in enumerate(np.unique(labels)):
#             if label1 != label2:
#                 cluster1_points = X[labels == label1]
#                 cluster2_points = X[labels == label2]
#                 # Compute pairwise distances between two clusters
#                 dist = np.min(pairwise_distances(np.concatenate([cluster1_points, cluster2_points])))
#                 inter_cluster_distances.append(dist)
    
#     # If no inter-cluster distances were computed, return NaN
#     if not inter_cluster_distances:
#         return np.nan

#     max_inter_cluster_distance = np.max(inter_cluster_distances)

#     # Avoid division by zero by checking if the max inter-cluster distance is greater than zero
#     if max_inter_cluster_distance == 0:
#         return np.nan

#     return min_intra_cluster_distance / max_inter_cluster_distance

def compute_dunn_index(X, labels):
    unique_clusters = np.unique(labels)
    num_clusters = len(unique_clusters)
    max_intracluster_dist = 0.0
    min_intercluster_dist = np.inf
    # Calculate maximum intracluster distance
    for i in unique_clusters:
        cluster_i_points = X[labels == i]
        # Handle clusters with single or no data points
        if len(cluster_i_points) <= 1:
            intracluster_dist = 0  # or any other appropriate value
        else:
            intracluster_dist = np.max(pdist(cluster_i_points))  
       
        if intracluster_dist > max_intracluster_dist:
            max_intracluster_dist = intracluster_dist

    # Calculate minimum intercluster distance
    for i in range(num_clusters):
        for j in range(i + 1, num_clusters):
            cluster_i_points = X[labels == unique_clusters[i]]
            cluster_j_points = X[labels == unique_clusters[j]]
            intercluster_dist = np.min(cdist(cluster_i_points, cluster_j_points))
            if intercluster_dist < min_intercluster_dist:
                min_intercluster_dist = intercluster_dist

    # Calculate Dunn index
    if max_intracluster_dist == 0:  # Handle case where all clusters have single points
        return 0  # or any other appropriate value
    else:
        return min_intercluster_dist / max_intracluster_dist
    
def compute_betacv(X, model):
    if isinstance(model, KMeans):
        labels = model.labels_
    elif isinstance(model, GaussianMixture):
        labels = model.predict(X)
    elif isinstance(model, KMedoids):
        labels = model.labels_

    # Calculate pairwise distances
    pairwise_dists = pairwise_distances(X)
    
    # Calculate intra-cluster distances (average distance within the same cluster)
    intra_cluster_distances = []
    for label in np.unique(labels):
        cluster_points = X[labels == label]
        intra_cluster_distances.append(np.mean(pairwise_distances(cluster_points)))
    
    # Calculate inter-cluster distances (average distance between different clusters)
    inter_cluster_distances = []
    for i, label1 in enumerate(np.unique(labels)):
        for j, label2 in enumerate(np.unique(labels)):
            if label1 != label2:
                cluster1_points = X[labels == label1]
                cluster2_points = X[labels == label2]
                inter_cluster_distances.append(np.mean(pairwise_distances(np.concatenate([cluster1_points, cluster2_points]))))
    
    # BetaCV is the ratio of intra-cluster distance mean to inter-cluster distance mean
    beta_cv = np.mean(intra_cluster_distances) / np.mean(inter_cluster_distances) if np.mean(inter_cluster_distances) != 0 else np.inf
    return beta_cv

def compute_hubert_statistic(X, model):
    # Get cluster labels
    if hasattr(model, 'labels_'):
        labels = model.labels_
    elif hasattr(model, 'predict'):
        labels = model.predict(X)
    else:
        raise ValueError("Unsupported clustering model")
    
    # Compute pairwise distances
    dist_matrix = squareform(pdist(X))
    
    # Create binary similarity matrix
    n = X.shape[0]
    similarity_matrix = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            similarity_matrix[i,j] = 1 if labels[i] == labels[j] else 0
    
    # Extract upper triangular indices
    mask = np.triu_indices(n, k=1)
    
    # Compute Pearson correlation
    dist_vector = dist_matrix[mask]
    similarity_vector = similarity_matrix[mask]
    
    # Compute correlation
    correlation, _ = pearsonr(dist_vector, similarity_vector)
    
    return correlation

def plot_clusters(X, model, model_name, image_path):
    # Reduce data to 2D for visualization
    pca = PCA(n_components=2)
    X_2d = pca.fit_transform(X)
    
    labels = None
    if model_name == "K-Means":
        centers = model.cluster_centers_
        labels = model.labels_
    elif model_name == "K-Medoids":
        centers = model.cluster_centers_
        labels = model.labels_
    elif model_name == "EM-Clustering (GMM)":
        # For GMM, the centers are the means of the Gaussian components
        centers = model.means_
        labels = model.predict(X)

    k = len(centers)
    
    plt.figure(figsize=(8, 6))
    # sns.scatterplot(x=X_2d[:, 0], y=X_2d[:, 1], hue=model.labels_, palette="Set2", s=100, alpha=0.7, edgecolor='black')
    scatter = sns.scatterplot(
        x=X_2d[:, 0], 
        y=X_2d[:, 1], 
        hue=labels,
        palette="Set2", 
        s=100, 
        alpha=0.7, 
        edgecolor='black'
    )

    # Project centers to the 2D PCA space
    centers_2d = pca.transform(centers)

    # plt.scatter(centers_2d[:, 0], centers_2d[:, 1], c='red', marker='X', s=200, label="Cluster Centers")
    plt.scatter(
        centers_2d[:, 0], 
        centers_2d[:, 1], 
        c='red', 
        marker='X', 
        s=200, 
        label="Cluster Centers"
    )
    
    plt.title(f"{model_name} Clustering with Cluster Centers")
    plt.xlabel("Principal Component 1")
    plt.ylabel("Principal Component 2")
    plt.legend(title="Clusters", title_fontsize=12, loc="best", fontsize=10)
    plt.grid(True, alpha=0.5)
    
    if os.path.exists(image_path) == False:
        os.makedirs(image_path)
    plot_name = os.path.join(image_path, f"{model_name}_clustering_{k}.png")
    print(f"Cluster plot saved as: {plot_name}")
    plt.savefig(plot_name)
    plt.close()

In [ ]:
# Function to perform and evaluate all clustering algorithms
def clustering_analysis(X, k, image_path):
    results = []

    # K-means
    kmeans = k_means_clustering(X, k)
    sse_kmeans = compute_sse(X, kmeans)
    silhouette_kmeans = compute_silhouette(X, kmeans)
    dunn_kmeans = compute_dunn_index(X, kmeans.labels_)
    hubert_kmeans = compute_hubert_statistic(X, kmeans)
    betacv_kmeans = compute_betacv(X, kmeans)
    plot_clusters(X, kmeans, "K-Means", image_path)

    results.append({
        "Model": "K-Means",
        "K": k,
        "SSE": sse_kmeans,
        "Silhouette": silhouette_kmeans,
        "Dunn's Index": dunn_kmeans,
        "Hubert's Statistic": hubert_kmeans,
        "BetaCV": betacv_kmeans
    })

    # EM-Clustering (GMM)
    gmm = em_clustering(X, k)
    labels = gmm.predict(X)
    sse_gmm = compute_sse(X, gmm)
    silhouette_gmm = compute_silhouette(X, gmm)
    dunn_gmm = compute_dunn_index(X, labels)
    hubert_gmm = compute_hubert_statistic(X, gmm)
    betacv_gmm = compute_betacv(X, gmm)
    plot_clusters(X, gmm, "EM-Clustering (GMM)", image_path)

    results.append({
        "Model": "EM-Clustering (GMM)",
        "K": k,
        "SSE": sse_gmm,
        "Silhouette": silhouette_gmm,
        "Dunn's Index": dunn_gmm,
        "Hubert's Statistic": hubert_gmm,
        "BetaCV": betacv_gmm
    })

    # K-medoids
    kmedoids_instance = k_medoids_clustering(X, k)
    sse_kmedoids = compute_sse(X, kmedoids_instance)
    silhouette_kmedoids = compute_silhouette(X, kmedoids_instance)
    dunn_kmedoids = compute_dunn_index(X, kmedoids_instance.labels_)
    hubert_kmedoids = compute_hubert_statistic(X, kmedoids_instance)
    betacv_kmedoids = compute_betacv(X, kmedoids_instance)
    plot_clusters(X, kmedoids_instance, "K-Medoids", image_path)

    results.append({
        "Model": "K-Medoids",
        "K": k,
        "SSE": sse_kmedoids,
        "Silhouette": silhouette_kmedoids,
        "Dunn's Index": dunn_kmedoids,
        "Hubert's Statistic": hubert_kmedoids,
        "BetaCV": betacv_kmedoids
    })

    return pd.DataFrame(results)

In [ ]:
# Training the models on original dataset

results_original_2 = clustering_analysis(X_scaled, 2, "./original_clustering_images")
results_original_4 = clustering_analysis(X_scaled, 4, "./original_clustering_images")
results_original_all = pd.concat([results_original_2, results_original_4], ignore_index=True)

In [ ]:
print("Clustering Analysis on Original Dataset\n")
print(tabulate(results_original_all, headers="keys", tablefmt="grid"))

In [ ]:
# Training the models on PCA dataset

results_pca_2 = clustering_analysis(X_train_pca, 2, "./pca_clustering_images")
results_pca_4 = clustering_analysis(X_train_pca, 4, "./pca_clustering_images")
results_pca_all = pd.concat([results_pca_2, results_pca_4], ignore_index=True)

In [ ]:
print("Clustering Analysis on PCA Dataset\n")
print(tabulate(results_pca_all, headers="keys", tablefmt="grid"))